In [18]:
from PIL import Image

from tensorflow.keras.models import load_model
import numpy as np
from numpy import asarray
from numpy import expand_dims
from keras_facenet import FaceNet
import pickle
import cv2

In [19]:
#HaarCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
HaarCascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))
MyFaceNet = FaceNet()

In [20]:
myfile = open("data.pkl", "rb")
database = pickle.load(myfile)
myfile.close()

In [22]:
cap = cv2.VideoCapture(0)

while(1):
    _, gbr1 = cap.read()
    
    wajah = HaarCascade.detectMultiScale(gbr1, 1.1, 4)
    
    if len(wajah) > 0:
        x1, y1, width, height = wajah[0] 
        x1, y1, x2, y2 = x1, y1, x1 + width, y1 + height  # Updated this line
        
        x1, y1 = abs(x1), abs(y1)
    
        gbr = cv2.cvtColor(gbr1, cv2.COLOR_BGR2RGB)
        gbr = Image.fromarray(gbr)                  
        gbr_array = asarray(gbr)
        
        face = gbr_array[y1:y2, x1:x2]                        
        
        face = Image.fromarray(face)                       
        face = face.resize((160, 160))
        face = asarray(face)
        face = expand_dims(face, axis=0)
        signature = MyFaceNet.embeddings(face)
        
        min_dist = 100
        identity = 'Unknown'  # Default to 'Unknown'
        
        for key, value in database.items():
            dist = np.linalg.norm(value - signature)
            if dist < min_dist:
                min_dist = dist
                identity = onlychar(key)
        
        # Check if the distance is above a certain threshold
        threshold = 1.0  # You can adjust this threshold based on your needs
        if min_dist > threshold:
            identity = 'Unknown'
            print(signature)
            color = (0, 0, 255)  # Red for 'Unknown'
        else:
            color = (0, 255, 0)  # Green for known identities
        
        cv2.putText(gbr1, identity, (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv2.LINE_AA)
        cv2.rectangle(gbr1, (x1, y1), (x2, y2), color, 2)
    
    cv2.imshow('res', gbr1)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cv2.destroyAllWindows()
cap.release()


1/1 [==============================] - 0s 66ms/step
[[ 0.02846077 -0.07374407  0.00048129  0.08540775 -0.00059671  0.05370166
  -0.07335526  0.01981164 -0.06949433 -0.00987232  0.00709381  0.06413597
   0.00261707 -0.0316143   0.0035911   0.05265005  0.05762557  0.02523433
   0.0294228  -0.04437842  0.01681058 -0.02703759 -0.00916036  0.05255517
   0.01210418 -0.00014697  0.02256794  0.04099832  0.00704232  0.08638328
  -0.05265446 -0.09592848  0.00217622 -0.05403058 -0.0272693  -0.00976445
  -0.03366747 -0.02422089 -0.01949677  0.0283826   0.00582129  0.04733658
  -0.02211969 -0.0327149  -0.00423189  0.07290628  0.07503409  0.01096691
  -0.00406305  0.05570291  0.01537419  0.03278005  0.04452708  0.02566027
  -0.02236994  0.00013608  0.08383462  0.0201808  -0.00790752  0.02305428
  -0.00922872  0.03481736 -0.0141061  -0.00945224 -0.00249503  0.10166313
   0.08176563 -0.01698777  0.04525874 -0.02986534  0.05693115  0.0815909
  -0.007592    0.04215465 -0.08238405  0.08191034 -0.04198856

1/1 [==============================] - 0s 63ms/step
[[-0.00684571 -0.03695223  0.02774352  0.09752486  0.0291387   0.04831243
  -0.05312109  0.00592717 -0.05869648 -0.01766291  0.00293561  0.015862
   0.01598488 -0.03173356  0.00242581  0.03842021  0.01359416  0.03577287
   0.00058384 -0.0615604  -0.01469446 -0.0081152  -0.00299089  0.03779732
   0.00049493  0.03948346  0.00735763 -0.01775569  0.03100603  0.06440789
  -0.08264559 -0.04894432 -0.00804462 -0.06497694 -0.01919653  0.02857362
   0.0011438  -0.03222439 -0.05792882  0.04230912  0.00297215  0.05656673
  -0.00624352 -0.03403865 -0.03008159  0.05266377  0.06883828  0.06685631
  -0.0435883   0.03150427  0.00382305  0.02692254  0.04060888  0.04409464
  -0.00014994  0.00706937  0.1090984   0.00040814 -0.00014811  0.00785919
   0.01552701  0.00875627 -0.00598553 -0.04347893 -0.02846299  0.08199858
   0.02828084  0.00692161  0.03175995 -0.02041808  0.04746009  0.07715036
   0.00292518  0.0119878  -0.04829636  0.0445129  -0.06734163 

In [23]:
import os
import re

directory_path = "anchor/"  # Replace with the path to your pictures directory
new_name = "Shepherd Umanah"

# Get a list of all files with the .jpg extension in the directory
jpg_files = [file for file in os.listdir(directory_path) if file.endswith(".jpg")]

# Rename each file with your name and a number
for i, old_name in enumerate(jpg_files, start=1):
    new_name_with_number = f"{new_name}_{i}.jpg"
    old_file_path = os.path.join(directory_path, old_name)
    new_file_path = os.path.join(directory_path, new_name_with_number)
    os.rename(old_file_path, new_file_path)

In [24]:
def onlychar(input_string):
    """
    Removes non-alphabetic characters from a string and returns the result.

    Parameters:
    input_string (str): The input string.

    Returns:
    str: The input string with only alphabetic characters.
    """
    result_string = ''.join(char for char in input_string if char.isalpha())
    return result_string